## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-02-16-46-09 +0000,nypost,House Dem leader rails that Mamdani isn’t futu...,https://nypost.com/2025/11/02/us-news/house-de...
1,2025-11-02-16-40-00 +0000,startribune,Live: J.J. McCarthy returns for the Vikings ag...,https://www.startribune.com/minnesota-vikings-...
2,2025-11-02-16-34-04 +0000,bbc,"Louvre heist carried out by petty criminals, p...",https://www.bbc.com/news/articles/c4gw42j29lqo...
3,2025-11-02-16-28-33 +0000,nyt,"California Promised Insurance Relief, But Deli...",https://www.nytimes.com/2025/11/01/us/los-ange...
4,2025-11-02-16-23-12 +0000,nyt,"Fire in Hermosillo, Mexico, Kills at Least 23 ...",https://www.nytimes.com/2025/11/01/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,24
152,train,11
50,new,9
151,stabbing,8
43,up,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
132,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...,57
46,2025-11-02-11-26-41 +0000,nypost,White House unveils details of ‘historic’ trad...,https://nypost.com/2025/11/02/us-news/donald-t...,54
117,2025-11-02-00-19-33 +0000,wapo,Trump threatens potential military action in N...,https://www.washingtonpost.com/world/2025/11/0...,54
157,2025-11-01-18-25-24 +0000,nypost,Nicki Minaj sends lefty fans into meltdown aft...,https://nypost.com/2025/11/01/us-news/nicki-mi...,51
12,2025-11-02-15-49-00 +0000,wsj,Trump Threatens Military Action in Nigeria in ...,https://www.wsj.com/world/africa/trump-threate...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
132,57,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...
27,50,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...
137,38,2025-11-01-21-04-33 +0000,nypost,NJ Dem chair admits ‘New Jersey is a battlegro...,https://nypost.com/2025/11/01/us-news/nj-dem-b...
42,33,2025-11-02-11-52-36 +0000,nypost,Two Louvre heist suspects had prior theft conv...,https://nypost.com/2025/11/02/world-news/louvr...
138,33,2025-11-01-20-55-31 +0000,nypost,House Republicans exploring ways to prevent Ma...,https://nypost.com/2025/11/01/us-news/house-re...
134,33,2025-11-01-21-27-14 +0000,latimes,"As SNAP benefits lapse, thousands show up to S...",https://www.latimes.com/california/story/2025-...
117,30,2025-11-02-00-19-33 +0000,wapo,Trump threatens potential military action in N...,https://www.washingtonpost.com/world/2025/11/0...
11,23,2025-11-02-15-49-03 +0000,bbc,Three killed in latest US strike on alleged dr...,https://www.bbc.com/news/articles/cn401vzjx48o...
122,23,2025-11-01-23-39-02 +0000,bbc,300 million tourists just visited China's stun...,https://www.bbc.com/news/articles/c8x1rnnd5gjo...
109,23,2025-11-02-01-39-48 +0000,nypost,Cuomo praises Adams plan to hire 5K more NYPD ...,https://nypost.com/2025/11/01/us-news/cuomo-pr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
